# Assignment 3: Creating a WordyPy Solver

You now understand the rules of _WordyPy_ and how to read the game state from an image. Now your job is to read in a
_WordyPy_ partial play and provide a next **good** guess. What's a good guess? A good guess is one which:

1. Continues to adhere to the rules of _WordPy_
2. Does not repeat words which have already been played
3. Uses the knowledge of previous guesses to pick a new good word

Unlike previous assignments there are no guardrails for this task -- you can complete this using whatever software
architecture you like! In addition, I've put my code in a new module for you to import, called `wordy`. You can just
import this module and begin making calls to it. You should be able to understand how it works by reading the module
documentation.


In [57]:
import wordy
import PIL
import pytesseract as pyt
import numpy as np
import random

In [15]:
from PIL import ImageDraw, Image

In [16]:
'''#import wordy
#import PIL
#ABHILASHA

def solution(board: PIL.Image) -> str:
    """The student solution to the problem.

    You must write code to query the wordy module and make
    a guess for the word. You needs to inspect the module to
    understand how to do this, and this function should only return
    the guess that you are going to make based on the game board state.

    Returns:b
        str: The guess that you are going to make.
    """
    
    new_guess = ""
#    config='-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    
    # YOUR CODE HERE
    
    copied = board.copy()
    w, h = copied.size
    rows = h//60
    print(f"your image has {rows} rows, and height is {h}, width is {w}")
    top_l1 = 0
    top_l2 = 0
    h=0
    for i in range(rows):
        t = (top_l1, top_l2 ,w,h+60)
        cropped = board.crop(t)
        #top_l1+=60
        top_l2+=60
        h+=60
        
        
#        cropped = cropped.resize((cropped.width//2, cropped.height*3))
        cropped = cropped.convert("L")
        drawing_object = ImageDraw.Draw(cropped)
        rect1 = (60,0,65,60)
        rect2 = (125,0,130,60)
        rect3 = (190,0,195,60)
        rect4 = (255,0,260,60)
        drawing_object.rectangle(rect1, fill=100)
        drawing_object.rectangle(rect2, fill=100)
        drawing_object.rectangle(rect3, fill=100)
        drawing_object.rectangle(rect4, fill=100)
    
        new_image = bytearray(cropped.tobytes())
        thres = 255
        for location, value in enumerate(new_image):
            if value < thres:
                new_image[location] = 0
            else:
                new_image[location] = 255
        new_image = PIL.Image.frombytes(cropped.mode,cropped.size,bytes(new_image))
        new_image = new_image.resize((new_image.width//2, new_image.height))
        text = pyt.image_to_string(new_image)
#        text = pyt.image_to_string(new_image, config='-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ' )
        
        display(cropped)
        print('\n')
        display(new_image)
        print('\n')
        
        print(text)
        
#        print(t)
#        print('\n')
        

        
            
        
        
#    copied = copied.resize((copied.width*1.2, copied.height*1.2))
#    copied = copied.convert("L")
#    new_image = bytearray(copied.tobytes())
#    thres = 255
#    for location, value in enumerate(new_image):
#        if value < thres:
#            new_image[location] = 0
#        else:
#           new_image[location] = 255
#    new_image = PIL.Image.frombytes(copied.mode,copied.size,bytes(new_image))
#    text = pyt.image_to_string(new_image, lang='eng')
#    display(board)
#    display(new_image)
    #text = pyt.image_to_string(board)
#    print("printing your text in next line")
#    print(text)
    #raise NotImplementedError()
    return new_guess'''

In [17]:
wl = wordy.get_word_list()
#wl

In [18]:
wordy.DisplaySpecification.block_height

60

In [19]:
# Create the implementation of the Letter class here
class Letter:
    """class Letter indicates a single English letter from a guess, 
    and has methods and attributes to describe whether the letter was in (or in the correct place) in the hidden target word.
   """
    def __init__(self, letter:str)->None:
        self.letter = letter
        self.in_correct_place:bool = False
        self.in_word:bool = False
    def is_in_correct_place(self)->bool:
        return self.in_correct_place
        
    def is_in_word(self)->bool:
        return self.in_word
        

In [62]:
def _tuple_to_str( pixels:tuple)->str:
        hex_ = '#%02x%02x%02x' % pixels
        return hex_.upper()
    
def _process_image(guess:str, guess_image:Image)->list[Letter]:
    letters = []
    stride = wordy.DisplaySpecification.block_width + wordy.DisplaySpecification.space_between_letters
    start = 0
    coordinate = 0
    #print(guess_image.size)
    #display(guess_image)
    #print(guess_image.getpixel((0,0)))
    #print(f"-------{guess}--------")
    for i in range(len(guess)):
        #print(i)
        letter_ = Letter(guess[i])
        #print("pixel:",guess_image.getpixel((coordinate,start)))
            
        pixel_hex = _tuple_to_str(guess_image.getpixel((coordinate,start)))
        #print(pixel_hex)
        if pixel_hex == wordy.DisplaySpecification.correct_location_color:
            letter_.in_correct_place = True
            letter_.in_word = True
        elif pixel_hex == wordy.DisplaySpecification.incorrect_location_color:
            letter_.in_word = True
                
        letters.append(letter_)
        #print(letter_.is_in_correct_place(), letter_.is_in_word())
        coordinate+=stride
    return letters     
     
            
def record_guess_results(guess:str, guess_results:Image, list_of_words:list)->list:
    #print(guess_results.size)
    guess_letters = _process_image(guess,guess_results)
       
    if guess in list_of_words:
        list_of_words.remove(guess)
    
    for ind, elem in enumerate(guess_letters):
            #l = str(elem.letter)
            #print("your letter is:", l)
            #ind = guess.index(l)
        if not elem.is_in_word(): # elem.is_in_word() == False
            list_of_words = [word for word in list_of_words if elem.letter not in word]
             
        if elem.is_in_correct_place()==True:
            list_of_words = [word for word in list_of_words if (elem.letter in word and word[ind] == elem.letter)]
    return list_of_words


In [78]:
#import wordy
#import PIL
#ABHILASHA

def solution(board: PIL.Image) -> str:
    """The student solution to the problem.

    You must write code to query the wordy module and make
    a guess for the word. You needs to inspect the module to
    understand how to do this, and this function should only return
    the guess that you are going to make based on the game board state.

    Returns:b
        str: The guess that you are going to make.
    """
    
    new_guess = ""
#    config='-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    
    # YOUR CODE HERE
    wl = wordy.get_word_list()
    copied = board.copy()
    w, h = copied.size
    rows = h//60
    #print(f"your image has {rows} rows, and height is {h}, width is {w}")
    top_l1 = 0
    top_l2 = 0
    h=0
    words = []
    for i in range(rows):
        t = (top_l1, top_l2 ,w,h+60)
        cropped = board.crop(t)
        #top_l1+=60
        top_l2+=60
        h+=60
        
        image = cropped.copy()
#        cropped = cropped.resize((cropped.width//2, cropped.height*3))
        cropped = cropped.convert("L")
        drawing_object = ImageDraw.Draw(cropped)
        rect1 = (60,0,65,60)
        rect2 = (125,0,130,60)
        rect3 = (190,0,195,60)
        rect4 = (255,0,260,60)
        drawing_object.rectangle(rect1, fill=100)
        drawing_object.rectangle(rect2, fill=100)
        drawing_object.rectangle(rect3, fill=100)
        drawing_object.rectangle(rect4, fill=100)
    
        new_image = bytearray(cropped.tobytes())
        thres = 255
        for location, value in enumerate(new_image):
            if value < thres:
                new_image[location] = 0
            else:
                new_image[location] = 255
        new_image = PIL.Image.frombytes(cropped.mode,cropped.size,bytes(new_image))
        new_image = new_image.resize((new_image.width//2, new_image.height))
        #text = pyt.image_to_string(new_image)
        text = pyt.image_to_string(new_image, config='-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ' )
        print(text)
        print(text.strip())
        print(len(text.strip()))
        wl = record_guess_results(text.strip(), image, wl)
        #display(cropped)
        #print('\n')
        #display(new_image)
        #print('\n')
        
        #print(text)
        
        
#        print(t)
#        print('\n')
        

        
            
        
        
#    copied = copied.resize((copied.width*1.2, copied.height*1.2))
#    copied = copied.convert("L")
#    new_image = bytearray(copied.tobytes())
#    thres = 255
#    for location, value in enumerate(new_image):
#        if value < thres:
#            new_image[location] = 0
#        else:
#           new_image[location] = 255
#    new_image = PIL.Image.frombytes(copied.mode,copied.size,bytes(new_image))
#    text = pyt.image_to_string(new_image, lang='eng')
#    display(board)
#    display(new_image)
    #text = pyt.image_to_string(board)
#    print("printing your text in next line")
#    print(text)
    #raise NotImplementedError()
    return random.choice(wl).upper()

In [41]:
#import wordy
#import PIL

'''
def solution(board: PIL.Image) -> str:
    """The student solution to the problem.

    You must write code to query the wordy module and make
    a guess for the word. You needs to inspect the module to
    understand how to do this, and this function should only return
    the guess that you are going to make based on the game board state.

    Returns:b
        str: The guess that you are going to make.
    """
    
    new_guess = ""
#    config='-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    
    # YOUR CODE HERE
    
    copied = board.copy()
    w, h = copied.size
    rows = h//60
    print(f"your image has {rows} rows, and height is {h}, width is {w}")
    top_l1 = 0
    top_l2 = 0
    w_stride = 0
    for i in range(5):
        t = (top_l1, top_l2 ,w_stride+60,h)
        cropped = board.crop(t)
        #top_l1+=60
        top_l1+=65
        w_stride+=65
        
        cropped = cropped.convert("L")
        cropped_arr = np.array(cropped)
        cropped_arr = cropped_arr.transpose()
        cropped = PIL.Image.fromarray(cropped_arr)
        #cropped = cropped.transpose(PIL.Image.Transpose.TRANSPOSE).transpose(PIL.Image.Transpose.ROTATE_180)
        #cropped = cropped.rotate(90)
        new_image = bytearray(cropped.tobytes())
        thres = 255
        for location, value in enumerate(new_image):
            if value < thres:
                new_image[location] = 255
            else:
                new_image[location] = 0
        new_image = PIL.Image.frombytes(cropped.mode,cropped.size,bytes(new_image))
        text = pyt.image_to_string(new_image, config='-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ' )
        
        display(cropped)
        print('\n')
        
        print(text)
        
#        print(t)
#        print('\n')
        

        
            
        
        
#    copied = copied.resize((copied.width*1.2, copied.height*1.2))
#    copied = copied.convert("L")
#    new_image = bytearray(copied.tobytes())
#    thres = 255
#    for location, value in enumerate(new_image):
#        if value < thres:
#            new_image[location] = 0
#        else:
#           new_image[location] = 255
#    new_image = PIL.Image.frombytes(copied.mode,copied.size,bytes(new_image))
#    text = pyt.image_to_string(new_image, lang='eng')
#    display(board)
#    display(new_image)
    #text = pyt.image_to_string(board)
#    print("printing your text in next line")
#    print(text)
    #raise NotImplementedError()
    return new_guess
    '''

'\ndef solution(board: PIL.Image) -> str:\n    """The student solution to the problem.\n\n    You must write code to query the wordy module and make\n    a guess for the word. You needs to inspect the module to\n    understand how to do this, and this function should only return\n    the guess that you are going to make based on the game board state.\n\n    Returns:b\n        str: The guess that you are going to make.\n    """\n    \n    new_guess = ""\n#    config=\'-c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ\'\n    \n    # YOUR CODE HERE\n    \n    copied = board.copy()\n    w, h = copied.size\n    rows = h//60\n    print(f"your image has {rows} rows, and height is {h}, width is {w}")\n    top_l1 = 0\n    top_l2 = 0\n    w_stride = 0\n    for i in range(5):\n        t = (top_l1, top_l2 ,w_stride+60,h)\n        cropped = board.crop(t)\n        #top_l1+=60\n        top_l1+=65\n        w_stride+=65\n        \n        cropped = cropped.convert("L")\n        cropped_arr = np.arr

In [86]:
# The autograder for this assignment is easy, it will try and play
# a few rounds of the game and ensure that errors are not thrown. If
# you can make it through five rounds we'll assume you have the right
# solution!
#
# You SHOULD NOT change anything in the wordy module, instead you
# must figure out how to write the solution() function in this notebook
# to make a good guess based on the board state!

for i in range(5):
    try:
        # Get an image of the current board state from wordy.
        # Note that the image contains some number of random guesses (always less than 5 guesses).
        image = wordy.get_board_state()
        # Create a new *good* guess based on the image and rules of wordy
        new_guess = solution(image)  # your code goes in solution()!
        # Send that guess to wordy to make sure it doesn't throw any errors
        wordy.make_guess(new_guess)
    except Exception as e:
        raise e

ASCUS

ASCUS
5
ARVAL

ARVAL
5
RAVEN

RAVEN
5
SAYER

SAYER
5
FLOSS

FLOSS
5
ROTTA

ROTTA
5
TODDY

TODDY
5
ELSIN

ELSIN
5
ROPER

ROPER
5
QUAVE

QUAVE
5
GAULT

GAULT
5
AMESS

AMESS
5
ALMRY

ALMRY
5
